In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [25]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest
from string import punctuation

In [26]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shivani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shivani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [28]:
import requests

In [29]:
url = "https://medium.com/ninjaconcept/interactive-dynamic-force-directed-graphs-with-d3-da720c6d7811"
headers = {'User-Agent':'Mozilla/5.0'}
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")


In [31]:
all_paragraphs = soup.find_all("p")

In [32]:
content = []

In [33]:
for x in all_paragraphs:
#     print(x.get_text()+"\n")
    content.append(x.get_text())

print(content[0])

Many companies have a tremendous amount of data which is not just hard to maintain and manage, but also hard to understand and analyse. Here, data visualisation should find a remedy. The main goal is to communicate information clearly and efficiently which improves the overall experience.


In [34]:
def findSummary(content):
    tokens = word_tokenize(content.lower())
    sentences = sent_tokenize(content)
    stop_words = set(stopwords.words('english') + list(punctuation))
    words = [word for word in tokens if (word.isalpha() and word not in stop_words)]
#     words[:10]
    word_freq = FreqDist(words)
    rankings = defaultdict(int)
    for i, sentence in enumerate(sentences):
        for word in word_tokenize(sentence.lower()):
            if word in word_freq:
                rankings[i] += word_freq[word]
#      rankings    
    indexes = nlargest(3, rankings, key=rankings.get)
    final_sentences = [sentences[j] for j in sorted(indexes)]
    summary = ' '.join(final_sentences)
    return summary

In [35]:
summary = []

In [ ]:
summary_i = findSummary(all_paragraphs[0])

In [ ]:
for para in all_paragraphs:
    summary_individual = findSummary(para.get_text())
    summary.append(summary_individual)
